In [20]:
from collections import OrderedDict
from torch.optim.lr_scheduler import StepLR
import torch
from torchvision.ops import nms
from torchvision.ops.boxes import box_iou
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2 
from torchvision.models.detection import fasterrcnn_resnet50_fpn 
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights, ssd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
import cv2
import random
import datetime
torch.manual_seed = 0
import sys
sys.path.append('..')
from conf import *
from inference.inference_helper import normalize_image, preprocess_image, inference_filter_prediction,denormalize_polygon
from ultralytics import YOLO
import numpy as np
from itertools import combinations, product

model_path1 = "../models/Good_FasterRcnn_V1_model.pth"
model_path2 = "../models/Good_FasterRcnn_V2_model.pth"
model_path3 = "../models/Good_SSD_model.pth"
yolo_path4 = "../models/yolov8.pt"

In [149]:

def load_model1(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path1)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path1} is loaded successfuly")
    return model

def load_model2(num_classes=NUMBER_OF_CLASSES):
    model = fasterrcnn_resnet50_fpn_v2(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    state_dict= torch.load(model_path2)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path2} is loaded successfuly")
    return model


def get_ssd_detection_model(num_classes=NUMBER_OF_CLASSES):
    ssd_model = ssd300_vgg16(weights=False)
    num_anchors = ssd_model.anchor_generator.num_anchors_per_location()
    out_channels = [512,1024,512,256,256,256]
    # ssd_model.head = ssd.SSDHead(out_channels, num_anchors, num_classes+1)
    state_dict= torch.load(model_path3)
    updated_state = {k.replace("module.", ""): v for k,v in state_dict.items()}
    ssd_model.load_state_dict(updated_state)
    print(f"MODEL from volume: {model_path3} is loaded successfuly")

    return ssd_model

def load_yolo():
    model = YOLO(yolo_path4)
    return model

def plot_boxes(normalized_road_roi_polygon, results, frame):
    """
    Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
    :param results: contains labels and coordinates predicted by model on the given frame.
    :param frame: Frame which has been scored.
    :return: Frame with bounding boxes and labels ploted on it.
    """
    label_bg_white = (255, 255, 255)
    if len(results) != 0:
        for result in results:
            for box, label,score in itertools.zip_longest(result['boxes'], result['labels'], result["scores"]):
                label = label.item()
                if label <= len(BOX_COLOR):
                    box_color = BOX_COLOR[label]
                    x1, x2, x3, x4 = int(box[0].item()), int(box[1].item()), int(box[2].item()), int(box[3].item())
                    cv2.rectangle(frame, (x1,x2),(x3,x4), box_color, 2)
                    if score is not None:
                        cv2.rectangle(frame, (x1, x2-25), (x1+150, x2), label_bg_white, -1)
                        label_text = f'{class_to_label(label)}: {score:.2f}'
                        cv2.putText(frame, label_text, (x1, x2 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)
    cv2.polylines(frame, [np.array(normalized_road_roi_polygon)], isClosed=True, color=(32, 32, 128), thickness=2)
    return frame

def class_to_label(label):
    return IDX_TO_CLASSES[label]

def save_test_img(img, target, prefix):
    # img = img.permute(2,0,1).cpu().numpy()  # Convert to (height, width, channels)

    # img = img.astype('uint8')
    # img = img
    # Draw bounding boxes on the image
    print(target)
    for box, label in zip(target['boxes'], target['labels']):
        x, y, w, h = box.tolist()
        x, y, w, h = int(x), int(y), int(w), int(h)
        # box_color = BOX_COLOR[label.item()]
        print(label)
        box_color = (255,255,255)
        cv2.rectangle(img, (x, y), (w, h), box_color, 2)

    # Save the image with bounding boxes
    if not os.path.exists(os.path.join(os.getcwd(), 'test_output')):
        os.makedirs(os.path.join(os.getcwd(), 'test_output'))
    # cv2.imshow(img)
    img_path = f"./test_output/output_image_{prefix}.png"
    cv2.imwrite(img_path, img)
    return img_path


def train(model, train_loader, optimizer, epoch):
    device = torch.device("cpu")
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader, 1):
        data = list(image.to(device) for image in data)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        optimizer.zero_grad()
        output = model(data, targets)
        print(f"=====[ epoch {epoch} batch {batch_idx}  output of the model: {output}")

        loss = output["loss_classifier"]
        loss.backward()
        optimizer.step()

def run(model, train_loader):
    epochs = 2
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

    for epoch in range(1, epochs + 1):
        train(model, train_loader, optimizer, epoch)
        scheduler.step()

def yolo_preprocess(img):
    # grayscale = to_grayscale(img)
    grayscale = img
    grayscale = normalize_image(grayscale)
    grayscale = torch.from_numpy(grayscale).float()
    grayscale = grayscale.unsqueeze(0)
    return grayscale
"""
To perform ensemble learning I have:
1. IoU to create intersection mask -> we get boxes which are predicted by both models
"""


def perform_ensemble_inference(predictions): # ((a,a,a) (b,b,b))
    final_prediction = []
    zipped_predictions = list(zip(*predictions)) # (a,b), (a,b), (a,b)
    print(len(predictions))
    print(len(zipped_predictions))

    for predictions_batch in zipped_predictions: # (a,b)
        print(len(predictions_batch))
        ensemble_voting(predictions_batch)
        # filter_common_predictions(predictions_batch)
    return final_prediction


def ensemble_voting(predictions_batch, iou_threshold=0.6):
    voting_result = {tuple(boxes): 1 for boxes in predictions_batch[0]["boxes"].tolist()}
    comparison_tensor = predictions_batch[0]["boxes"]
    print("voting_result: ", voting_result)
    for prediction in predictions_batch[1:]:
        boxes = prediction["boxes"]
        iou_mask = box_iou(comparison_tensor, boxes)
        mask1 = iou_mask.sum(axis=1) > iou_threshold
        mask2 = iou_mask.sum(axis=0) > iou_threshold

        print("comparison_tensor: ", comparison_tensor)
        print("boxes: ", boxes)

        print("mask1: ", mask1)
        print("mask2: ", mask2)

        matching_boxes = comparison_tensor[mask1]
        new_boxes = boxes[~mask2]
        print("matching_boxes: ", matching_boxes)
        print("new_boxes: ", new_boxes)
        for box in matching_boxes:
            box = tuple(box.tolist())
            if box in voting_result:
                voting_result[box] += 1
            else:
                voting_result[box] = 1
        if len(new_boxes):
            comparison_tensor = torch.cat((comparison_tensor, new_boxes))
    print("voting_result: ",voting_result)



        


def filter_common_predictions(predictions_batch, iou_threshold=0.6): # (pred1, pred2)
    num_of_predictions = len(predictions_batch)
    predictions_batch = {(key,): prediction["boxes"] for key, prediction in enumerate(predictions_batch)}
    
    print("predictions_batch: ", predictions_batch)
    iou_mask = compute_iou_for_all_pairs(predictions_batch, iou_threshold) 


    partial_common_boxes = {(*i, j): predictions_batch[tuple(i)][mask] for (*i,j), mask in iou_mask.items()}
    print("partial_common_boxes: ", partial_common_boxes)
    iou_mask = compute_iou_for_all_pairs(partial_common_boxes, iou_threshold) 
    print("iou_mask 2: ", iou_mask)
    partial_common_boxes = {(*i, j): partial_common_boxes[tuple(i)][mask] for (*i,j), mask in iou_mask.items()}
    print("partial_common_boxes 2: ", partial_common_boxes)
    iou_mask = compute_iou_for_all_pairs(partial_common_boxes, iou_threshold) 
    print("iou_mask 3: ", iou_mask)
    partial_common_boxes = {(*i, j): partial_common_boxes[tuple(i)][mask] for (*i,j), mask in iou_mask.items()}
    print("partial_common_boxes 3: ", partial_common_boxes)

def compute_iou_for_all_pairs(predictions_batch, iou_threshold=0.6):
    # Generate all pairs of indices for bounding_boxes_list
    pairs = list(combinations(range(len(predictions_batch)), 2))

    # Compute IoU for each pair of bounding box sets
    iou_matrices = {}
    for i, j in pairs:
        predictions = list(predictions_batch.items())
        key1, box1 = predictions[i]
        key2, box2 = predictions[j]
        intersecting_keys = len(set(key1).difference(key2))
        print("INTERSECTION: ", intersecting_keys)
        new_key = tuple(set(key1+key2))

        print(f"key1: {key1} box1 {box1}")
        print(f"key2: {key2} box2 {box2}")
        if len(box1) > 0 and len(box2) > 0 and intersecting_keys == 1 and not new_key in iou_matrices:
            iou_mask = box_iou(box1, box2).sum(axis=1)  > iou_threshold
            print(f"iou_mask: {iou_mask}")
            iou_matrices[new_key] = iou_mask
    print("==========================")
    print("iou_matrices", iou_matrices)
    return iou_matrices



In [22]:
model1 = load_model1()
model2 = load_model2()
model3 = get_ssd_detection_model()
# model4 = load_yolo()

model1.eval()
model2.eval()
model3.eval()
# model4.eval()

original_image_sizes = []

MODEL from volume: ../models/Good_FasterRcnn_V1_model.pth is loaded successfuly
MODEL from volume: ../models/Good_FasterRcnn_V2_model.pth is loaded successfuly
MODEL from volume: ../models/Good_SSD_model.pth is loaded successfuly


In [150]:
image_dir = "./real_images/simple.jpg"
image_dim = (960,1280)
denormalized_road_roi_polygon = denormalize_polygon(image_dim, ROAD_ROI_POLYGON)

image = cv2.imread(image_dir)
grayscale = preprocess_image(image)
grayscale = grayscale.unsqueeze(0)
inputs = grayscale

timer_model1 = datetime.datetime.now()
with torch.no_grad():
    outputs1 = model1(inputs)
timer_model1 = datetime.datetime.now() - timer_model1 

timer_model2 = datetime.datetime.now()
with torch.no_grad():
    outputs2 = model2(inputs)
timer_model2 = datetime.datetime.now() - timer_model2

timer_model3 = datetime.datetime.now()
with torch.no_grad():
    outputs3 = model3(inputs)
timer_model3 = datetime.datetime.now() - timer_model3

print("Model1 filtering: ")
outputs1 = inference_filter_prediction(outputs1, denormalized_road_roi_polygon,iou_threshold=0.3)
print("Model2 filtering: ")
outputs2 = inference_filter_prediction(outputs2, denormalized_road_roi_polygon)
print("Model3 filtering: ")
outputs3 = inference_filter_prediction(outputs3, denormalized_road_roi_polygon, iou_threshold=0.3)

final_prediction = perform_ensemble_inference([outputs3, outputs2, outputs1])

# print("final_prediction: ", final_prediction)
# print("outputs1: ", outputs1)
# print("outputs3: ", outputs3)

# """
# Common predictions are left without a change
# Model1 * 0.7 + Model2*0.3
# """


# it = random.randint(0, 1000)
# prefix=f"detections-{it}"
# frame1 = plot_boxes(denormalized_road_roi_polygon, outputs1, np.copy(image))
# frame3 = plot_boxes(denormalized_road_roi_polygon, outputs3, np.copy(image))
# ensemble_frame = plot_boxes(denormalized_road_roi_polygon, final_prediction, np.copy(image))

# for output1, output3, ensemble_pred in zip(outputs1,outputs3, final_prediction):
    
#     save_test_img(frame1, output1, f"FasterRcnn_{prefix}")
#     save_test_img(frame3, output3, f"SSD_{prefix}")
#     save_test_img(ensemble_frame, ensemble_pred, f"Ensemble_{prefix}")

# print(f"Model1: {timer_model1}\nModel3: {timer_model3}\n")


Model1 filtering: 
Filtered based on  polygon:  {'scores': tensor([0.9542, 0.9492, 0.9237, 0.9019]), 'labels': tensor([4, 4, 4, 4]), 'boxes': tensor([[305.3919, 513.8285, 384.6668, 600.6637],
        [364.9731, 704.9017, 591.7959, 971.8295],
        [316.7217, 457.6261, 384.8579, 490.2362],
        [311.6656, 372.6617, 364.1157, 421.6015]])}
AFTER CLEANING ON THE ROAD {'scores': tensor([0.9542, 0.9492, 0.9237, 0.9019]), 'labels': tensor([4, 4, 4, 4]), 'boxes': tensor([[305.3919, 513.8285, 384.6668, 600.6637],
        [364.9731, 704.9017, 591.7959, 971.8295],
        [316.7217, 457.6261, 384.8579, 490.2362],
        [311.6656, 372.6617, 364.1157, 421.6015]])}
Model2 filtering: 
Filtered based on  polygon:  {'scores': tensor([0.9391, 0.9262, 0.9215, 0.9159, 0.8990]), 'labels': tensor([4, 4, 4, 4, 4]), 'boxes': tensor([[ 301.0785,  517.4127,  391.3704,  599.2695],
        [ 353.1155,  679.5331,  581.2241,  961.9435],
        [ 316.4129,  446.1359,  388.3924,  495.4585],
        [ 363.5381

In [25]:
x = {(0,1): torch.tensor([[1,2,3],[3,4,5]]),
     (0,2): torch.tensor([[3,4,5],[3,3,3]]),
     (1,2): torch.tensor([[3,4,5],[4,4,4]])}

"output: (0,1,2,3): [3],(0,1):[1,2], (0,2):[4])"
merged_dict = {}

for ref_key, ref_values in x.items():
    merged_key = []
    for key, values in x.items():
        if ref_key[0] == key[0] and ref_key[1] != key[1]:
            mask = torch.eq(values, ref_values)
            print(f"values: {values}, \nref_values: {ref_values}")
            print(mask) 
            # mask = [True if item in ref_values else False for item in values]
            # print(f"ref_value: {ref_values}, values: {values}")
            # print(mask)



print("Output:", merged_dict)


values: tensor([[3, 4, 5],
        [3, 3, 3]]), 
ref_values: tensor([[1, 2, 3],
        [3, 4, 5]])
tensor([[False, False, False],
        [ True, False, False]])
values: tensor([[1, 2, 3],
        [3, 4, 5]]), 
ref_values: tensor([[3, 4, 5],
        [3, 3, 3]])
tensor([[False, False, False],
        [ True, False, False]])
Output: {}
